1. Candlestick Chart (Японские свечи)

Задания:

    Построение графика

        Возьми данные акций Tesla (TSLA) за последние 3 месяца.

        Построй свечной график с периодами 1 день.

        Добавь скользящую среднюю (MA) за 20 дней.


In [ ]:
import pandas as pd

tesla = pd.read_csv('/content/TSLA_250101_250331.csv', sep=';')
tesla.head()

,<TICKER>,<PER>,<DATE>,<TIME>,<OPEN>,<HIGH>,<LOW>,<CLOSE>,<VOL>
0,TSLA,D,250102,0,390.55,392.665,373.04,379.35,2022729
1,TSLA,D,250103,0,381.51,411.840,379.58,410.73,2326366
2,TSLA,D,250106,0,423.20,426.410,401.77,411.14,1511961
3,TSLA,D,250107,0,405.88,414.290,390.02,394.60,1446023
4,TSLA,D,250108,0,393.04,402.430,387.40,394.95,1449515


In [ ]:
import plotly.graph_objects as go
from datetime import datetime, timedelta

tesla['<DATE>'] = pd.to_datetime(tesla['<DATE>'], format='%y%m%d')
tesla = tesla[tesla['<DATE>'] > pd.to_datetime('2000-01-01')]  # фильтр дат после 2000 года

tesla = tesla.sort_values('<DATE>')
tesla.set_index('<DATE>', inplace=True)

# фильтрация последних 3 месяцев
last_date = tesla.index.max()
three_months_ago = last_date - pd.DateOffset(months=3)
tesla = tesla.loc[three_months_ago:last_date]

# расчет скользящей средней
tesla['MA20'] = tesla['<CLOSE>'].rolling(20).mean()

fig = go.Figure(data=[
    go.Candlestick(
        x=tesla.index,
        open=tesla['<OPEN>'],
        high=tesla['<HIGH>'],
        low=tesla['<LOW>'],
        close=tesla['<CLOSE>'],
        name='Свечи TSLA',
        increasing_line_color='green',
        decreasing_line_color='red'
    ),
    go.Scatter(
        x=tesla.index,
        y=tesla['MA20'],
        line=dict(color='blue', width=2),
        name='20-дневная MA'
    )
])

fig.update_layout(
    title=f'Акции Tesla (TSLA) за последние 3 месяца ({three_months_ago.date()} - {last_date.date()})',
    xaxis_title='Дата',
    yaxis_title='Цена ($)',
    xaxis_rangeslider_visible=False,
    template='plotly_white',
    hovermode='x unified'
)

fig.update_xaxes(
    rangeselector=dict(
        buttons=list([
            dict(count=1, label="1 мес", step="month", stepmode="backward"),
            dict(count=3, label="3 мес", step="month", stepmode="backward"),
            dict(count=6, label="6 мес", step="month", stepmode="backward"),
            dict(step="all", label="Все данные")
        ])
    )
)

fig.show()

2. Point and Figure Chart (Точечно-цифровая)

Задания:
Построение P&F графика

    Возьми данные по золоту (XAU/USD) за 2023 год.

    Построй график с шагом $10 и разворотом на 3 шага.

    Как должно выглядеть: Колонки X (рост) и O (падение) без временной оси.

In [ ]:
gold = pd.read_csv('/content/GC_230101_231231.csv', sep=';')
gold.head()

,<TICKER>,<PER>,<DATE>,<TIME>,<OPEN>,<HIGH>,<LOW>,<CLOSE>,<VOL>
0,GC,D,230101,0,1823.580,1823.58,1822.6,1823.530,9
1,GC,D,230102,0,1823.530,1830.73,1822.7,1829.390,16639
2,GC,D,230103,0,1829.308,1850.15,1825.9,1836.348,1124714
3,GC,D,230104,0,1836.460,1865.15,1835.3,1854.820,1131983
4,GC,D,230105,0,1854.720,1859.03,1824.0,1832.820,1037842


In [ ]:
gold['<DATE>'] = pd.to_datetime(gold['<DATE>'])
gold.sort_values('<DATE>', inplace=True)

# Параметры P&F
box_size = 10  # размер шага
reversal = 3   # разворот

# Алгоритм построения P&F
current_col = []
columns = []
direction = None
prev_price = gold.iloc[0]['<CLOSE>']

for price in gold['<CLOSE>']:
    if not direction:
        if price >= prev_price + box_size:
            direction = 'X'
            current_col = ['X'] * int((price - prev_price) / box_size)
        elif price <= prev_price - box_size:
            direction = 'O'
            current_col = ['O'] * int((prev_price - price) / box_size)
    else:
        if direction == 'X':
            if price >= prev_price + box_size:
                current_col.extend(['X'] * int((price - prev_price) / box_size))
            elif price <= prev_price - reversal * box_size:
                columns.append(current_col)
                direction = 'O'
                current_col = ['O'] * int((prev_price - price) / box_size)
        else:
            if price <= prev_price - box_size:
                current_col.extend(['O'] * int((prev_price - price) / box_size))
            elif price >= prev_price + reversal * box_size:
                columns.append(current_col)
                direction = 'X'
                current_col = ['X'] * int((price - prev_price) / box_size)
    prev_price = price

# Визуализация
fig = go.Figure()

for i, col in enumerate(columns):
    for j, val in enumerate(col):
        fig.add_trace(
            go.Scatter(
                x=[i, i],
                y=[j*box_size, (j+1)*box_size],
                mode='lines',
                line=dict(color='green' if val == 'X' else 'red', width=10),
                showlegend=False
            )
        )

fig.update_layout(
    title=f'Точечно-цифровой график золота (шаг ${box_size}, разворот {reversal})',
    xaxis=dict(visible=False),
    yaxis_title='Цена ($)',
    template='plotly_white'
)

fig.show()


3. Waterfall Chart (Водопадная диаграмма)

Задания:
Анализ прибыли компании

    Данные: Выручка = +500, Затраты = -300, Налоги = -100, Курсовая разница = +50.

    Построй водопад от начального баланса до чистой прибыли.

    Как должно выглядеть: Столбцы с подписями "+" и "-", итог – зеленый/красный.

        Как должно выглядеть: График с зелеными/красными свечами и синей линией MA.

In [ ]:
import plotly.graph_objects as go

# Данные
categories = ['Начало', 'Выручка', 'Затраты', 'Налоги', 'Курсовая', 'Итог']
values = [0, 500, -300, -100, 50, 150]  # Последнее значение - итог

colors = ['lightgray'] + ['green' if v > 0 else 'red' for v in values[1:-1]] + ['green' if values[-1] > 0 else 'red']

fig = go.Figure(go.Waterfall(
    name="Прибыль",
    orientation="v",
    x=categories,
    y=values,
    textposition="outside",
    text=[f"+{v}" if v > 0 else str(v) for v in values],
    connector={"line":{"color":"gray"}},
    increasing={"marker":{"color":"green"}},
    decreasing={"marker":{"color":"red"}},
    totals={"marker":{"color":"lightgray"}}
))

fig.update_layout(
    title="Анализ прибыли компании (водопадная диаграмма)",
    yaxis_title="Сумма ($)",
    showlegend=False,
    template='plotly_white'
)

fig.show()